<a href="https://colab.research.google.com/github/LeonardoRoig/TECH_5/blob/main/Vagas_pos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast, json, re
import warnings
warnings.filterwarnings('ignore')

# --- Nova célula ---

df_raw = pd.read_json("vagas.json")

# --- Nova célula ---

# 1) Garantir que cada célula seja um dict (pode vir como string)
def to_dict(x):
    if isinstance(x, dict):
        return x
    if pd.isna(x):
        return {}
    s = str(x)
    # tenta Python-literal
    try:
        return ast.literal_eval(s)
    except Exception:
        pass
    # tenta JSON
    try:
        return json.loads(s)
    except Exception:
        return {}

df_raw = df_raw.applymap(to_dict)

# --- Nova célula ---

# 2) Transpor: cada linha vira um job_id
df_t = df.T  # index = job_id; colunas = ['informacoes_basicas','perfil_vaga','beneficios']
df_t.index.name = "job_id"

# 3) Combinar os 3 dicionários por linha com prefixos para evitar colisão de nomes
def merge_sections(row):
    base = {f"inf_{k}": v for k, v in row.get("informacoes_basicas", {}).items()}
    perf = {f"perfil_{k}": v for k, v in row.get("perfil_vaga", {}).items()}
    benef = {f"benef_{k}": v for k, v in row.get("beneficios", {}).items()}
    return {**base, **perf, **benef}

# --- Nova célula ---

records = df_t.apply(merge_sections, axis=1)
df_flat = pd.DataFrame(list(records), index=df_t.index).reset_index()

# 4) Limpeza básica de colunas
df_flat.columns = (
    pd.Series(df_flat.columns)
    .astype(str)
    .str.strip()
    .str.lower()
    .str.normalize('NFKD')  # remove acentos
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace(r'\s+', '_', regex=True)
)

# --- Nova célula ---

# 5) Tipagem: datas (dia primeiro), valores monetários, inteiros
# Datas: qualquer coluna que contenha 'data'
for c in [c for c in df_flat.columns if 'data' in c]:
    df_flat[c] = pd.to_datetime(df_flat[c], dayfirst=True, errors='coerce')

# Função para converter dinheiro/bruto em float
money_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['valor', ...